In [2]:
import os
import shutil
import re
import pandas as pd
from pandas import DataFrame
from pathlib import Path
from icecream import ic
import logging
import time

In [3]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
now_times = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

In [4]:
dir_path= '/mnt/nas1/lanwei-125/FGF5/test/'
result_path = f'{dir_path}/refinement/'
protein_pdb = '/mnt/nas1/lanwei-125/FGF5/dock_prepare/FGF5.pdb'
protein_chain = "A"
peptide_chain = "E"

In [5]:
cluster_dir_names = [
    dir_name for dir_name in Path(dir_path).iterdir()
    if dir_name.is_dir() and dir_name.name.split("_")[0] == "cluster"
]

pdb_names = [
    file.stem for cluster_dir in cluster_dir_names
    for file in cluster_dir.iterdir() if file.is_file() and file.suffix == ".pdb"
    ]
[str(cluster_dir_name) for cluster_dir_name in cluster_dir_names]

['/mnt/nas1/lanwei-125/FGF5/test/cluster_0',
 '/mnt/nas1/lanwei-125/FGF5/test/cluster_1',
 '/mnt/nas1/lanwei-125/FGF5/test/cluster_2',
 '/mnt/nas1/lanwei-125/FGF5/test/cluster_3',
 '/mnt/nas1/lanwei-125/FGF5/test/cluster_4',
 '/mnt/nas1/lanwei-125/FGF5/test/cluster_5',
 '/mnt/nas1/lanwei-125/FGF5/test/cluster_6',
 '/mnt/nas1/lanwei-125/FGF5/test/cluster_-1']

E


In [6]:
protein_lines = []
protein_chain = set()
with open(protein_pdb) as f:
    protein_lines = [line for line in f if line.startswith("ATOM") or line.startswith("TER")]
    for line in protein_lines:
        if line.startswith('ATOM') :
            chains = line[21]
            protein_chain.add(chains)
protein_chain_str = ','.join(protein_chain)

peptide_chain = set()
for cluster_dir_name in cluster_dir_names:
    pdb_path = Path(dir_path) / cluster_dir_name

    for pdb_file in pdb_path.glob('*.pdb'):
        parts = pdb_file.stem.split("-")
        if len(parts) >= 2 and parts[1] == "ref":
            continue

        if pdb_file.suffix == '.pdb' and '-ref' not in pdb_file.stem:
            # Read peptide lines
            peptide_lines = []
            with open(pdb_file) as f:
                peptide_lines = [line for line in f if line.startswith("ATOM")]
                for line in peptide_lines:
                    if line.startswith('ATOM') :
                        chains = line[21]
                        peptide_chain.add(chains)
            peptide_chain_str = ','.join(peptide_chain)

            # Write to new pdb file
            os.makedirs(pdb_path / pdb_file.stem, exist_ok=True)
            new_pdb_path = pdb_path / pdb_file.stem / (pdb_file.stem + '-ref.pdb')
            with open(new_pdb_path, 'w') as f:
                f.write(''.join(protein_lines))
                if protein_lines[-1].startswith('END'):
                    #删除END语句
                    del protein_lines[-1]
                if not protein_lines[-1].startswith('TER'):
                    f.write("\nTER\n")
                f.write(''.join(peptide_lines))
                f.write("\nTER\n")
                f.write("END\n")

            logger.info(f'{now_times}: processing {pdb_file} to {new_pdb_path} ')
    

2023-12-19 13:58:33,080 - INFO - 2023-12-19 13:58:16: processing /mnt/nas1/lanwei-125/FGF5/test/cluster_test/CTSESDC.pdb to /mnt/nas1/lanwei-125/FGF5/test/cluster_test/CTSESDC/CTSESDC-ref.pdb


2023-12-19 13:58:33,128 - INFO - 2023-12-19 13:58:16: processing /mnt/nas1/lanwei-125/FGF5/test/cluster_test/CYIGSCVLG.pdb to /mnt/nas1/lanwei-125/FGF5/test/cluster_test/CYIGSCVLG/CYIGSCVLG-ref.pdb


In [7]:
np_nums = "12"
for cluster_dir_name in cluster_dir_names:

    for file in (Path(dir_path) / cluster_dir_name).glob('*.pdb'):
        pdb_file = file.stem
       
        work_dir = Path(dir_path) / cluster_dir_name / pdb_file
        os.chdir(work_dir)
        for file in work_dir.glob('*-ref.pdb'):
            ref_pdb = file.stem
            logger.info(f'{now_times}: processing {file}')

            f=open('prepack.flags', 'w')
            f.write("-s "+ ref_pdb + ".pdb \n")
            f.write("-flexpep_prepack \n")
            f.write("-nstruct 1 \n")
            f.write("-ex1 \n")
            f.write("-ex2aro \n")
            f.write("-mute core.io.database \n")
            f.write("-mute core.util.prof \n")
            f.write("-mute protocols.jd2.JobDistributor \n")
            f.write("-scorefile pack.score.sc ")
            f.close()
            os.system("mpirun -np "+ np_nums + " FlexPepDocking.mpi.linuxgccrelease @prepack.flags")

            f=open('refinement.flags', 'w')
            f.write("-s "+ ref_pdb + "_0001.pdb \n")
            #f.write("-native "+ peptide + ".pdb \n")
            f.write("-flexPepDocking:receptor_chain "+protein_chain+" \n")
            f.write("-flexPepDocking:peptide_chain "+peptide_chain+" \n")
            f.write("-pep_refine \n")
            f.write("-use_input_sc \n")
            f.write("-nstruct 250 \n")
            f.write("-scorefile refinement.score.sc \n")
            f.write("-ex1 \n")
            f.write("-ex2aro \n")
            f.write("-mute core.io.database \n")
            f.write("-mute core.util.prof \n")
            f.write("-mute protocols.jd2.JobDistributor ")
            f.close()
            os.system("mpirun.openmpi -np "+ np_nums + " FlexPepDocking.mpi.linuxgccrelease @refinement.flags")


2023-12-19 13:59:24,778 - INFO - 2023-12-19 13:58:16: processing /mnt/nas1/lanwei-125/FGF5/test/cluster_test/CTSESDC/CTSESDC-ref.pdb
No protocol specified


core.init: (7) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (7) Rosetta version: rosetta.binary.ubuntu.release-356 r356 2023.35+release.23439d33534 23439d33534e3f106f2ad301c8c3c56013ca8471 https://www.rosettacommons.org 2023-09-02T00:17:53.640884
core.init: (7) command: FlexPepDocking.mpi.linuxgccrelease @prepack.flags
core.init: (5) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (5) Rosetta version: rosetta.binary.ubuntu.release-356 r356 2023.35+release.23439d33534 23439d33534e3f106f2ad301c8c3c56013ca8471 https://www.rosettacommons.org 2023-09-02T00:17:53.640884
core.init: (5) command: FlexPepDocking.mpi.linuxgccrelease @prepack.flags
core.init: (11) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (6) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (11) Rosetta version: rosetta.binary.ubuntu.release-356 r356 2023.35+release.23439d33534 23439d33534e3f106f2ad301c8c3c56013ca8471 https://www.rosettacommons.org 202

No protocol specified


core.init: (10) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (10) Rosetta version: rosetta.binary.ubuntu.release-356 r356 2023.35+release.23439d33534 23439d33534e3f106f2ad301c8c3c56013ca8471 https://www.rosettacommons.org 2023-09-02T00:17:53.640884
core.init: (10) command: FlexPepDocking.mpi.linuxgccrelease @refinement.flags
core.init: (7) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (7) Rosetta version: rosetta.binary.ubuntu.release-356 r356 2023.35+release.23439d33534 23439d33534e3f106f2ad301c8c3c56013ca8471 https://www.rosettacommons.org 2023-09-02T00:17:53.640884
core.init: (7) command: FlexPepDocking.mpi.linuxgccrelease @refinement.flags
core.init: (8) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (8) Rosetta version: rosetta.binary.ubuntu.release-356 r356 2023.35+release.23439d33534 23439d33534e3f106f2ad301c8c3c56013ca8471 https://www.rosettacommons.org 2023-09-02T00:17:53.640884
core.init: (8) command: FlexPepDocking.

2023-12-19 13:59:31,206 - INFO - 2023-12-19 13:58:16: processing /mnt/nas1/lanwei-125/FGF5/test/cluster_test/CYIGSCVLG/CYIGSCVLG-ref.pdb
No protocol specified


core.init: (8) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (8) Rosetta version: rosetta.binary.ubuntu.release-356 r356 2023.35+release.23439d33534 23439d33534e3f106f2ad301c8c3c56013ca8471 https://www.rosettacommons.org 2023-09-02T00:17:53.640884
core.init: (8) command: FlexPepDocking.mpi.linuxgccrelease @prepack.flags
core.init: (0) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (0) Rosetta version: rosetta.binary.ubuntu.release-356 r356 2023.35+release.23439d33534 23439d33534e3f106f2ad301c8c3c56013ca8471 https://www.rosettacommons.org 2023-09-02T00:17:53.640884
core.init: (0) command: FlexPepDocking.mpi.linuxgccrelease @prepack.flags
basic.random.init_random_generator: (0) 'RNG device' seed mode, using '/dev/urandom', seed=-956425630 seed_offset=0 real_seed=-956425630
basic.random.init_random_generator: (0) RandomGenerator:init: Normal mode, seed=-956425630 RG_type=mt19937
basic.random.init_random_generator: (8) 'RNG device' seed mode, using '/

No protocol specified


core.init: (7) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (7) Rosetta version: rosetta.binary.ubuntu.release-356 r356 2023.35+release.23439d33534 23439d33534e3f106f2ad301c8c3c56013ca8471 https://www.rosettacommons.org 2023-09-02T00:17:53.640884
core.init: (2) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (2) Rosetta version: rosetta.binary.ubuntu.release-356 r356 2023.35+release.23439d33534 23439d33534e3f106f2ad301c8c3c56013ca8471 https://www.rosettacommons.org 2023-09-02T00:17:53.640884
core.init: (2) command: FlexPepDocking.mpi.linuxgccrelease @refinement.flags
core.init: (7) command: FlexPepDocking.mpi.linuxgccrelease @refinement.flags
core.init: (5) Checking for fconfig files in pwd and ./rosetta/flags 
core.init: (5) Rosetta version: rosetta.binary.ubuntu.release-356 r356 2023.35+release.23439d33534 23439d33534e3f106f2ad301c8c3c56013ca8471 https://www.rosettacommons.org 2023-09-02T00:17:53.640884
core.init: (5) command: FlexPepDocking.mpi

In [8]:
os.system("mpirun -h")

mpirun (Open MPI) 4.0.3

Usage: mpirun [OPTION]...  [PROGRAM]...
Start the given program using Open RTE

-c|-np|--np <arg0>       Number of processes to run
-h|--help <arg0>         This help message
   -n|--n <arg0>         Number of processes to run
-q|--quiet               Suppress helpful messages
-v|--verbose             Be verbose
-V|--version             Print version and exit

For additional mpirun arguments, run 'mpirun --help <category>'

The following categories exist: general (Defaults to this option), debug,
    output, input, mapping, ranking, binding, devel (arguments useful to OMPI
    Developers), compatibility (arguments supported for backwards compatibility),
    launch (arguments to modify launch options), and dvm (Distributed Virtual
    Machine arguments).

Report bugs to http://www.open-mpi.org/community/help/


0

In [27]:
result_path = Path(result_path)
if not result_path.exists():
    result_path.mkdir()


for file in (Path(dir_path) / cluster_dir_name).glob('*.pdb'):
    pdb_file = file.stem
    work_dir = Path(dir_path) / cluster_dir_name / pdb_file  
    score_file = work_dir / 'refinement.score.sc'

    with open(score_file, 'r') as f:
        headers = None
        rows = []
        lines = f.readlines()
        for line in lines:
            if line.startswith('SEQUENCE:'):
                continue
            elif line.startswith('SCORE:'):
                if headers is None:
                    headers = line.split()[1:]
                else:
                    rows.append(line.split()[1:])
            else:
                rows.append(line.split())
        df = pd.DataFrame(rows, columns=headers)
        df['reweighted_sc'] = df['reweighted_sc'].astype(float)
        df = df.sort_values(by='reweighted_sc', ignore_index=True)
        min_row = df.loc[df['reweighted_sc'].idxmin()]

    candidate_path = work_dir / f"{min_row['description']}.pdb"
    shutil.copy(candidate_path, result_path)

    result_score_file = result_path / 'candidate.txt'
    with open(result_score_file, 'a') as f:
        f.write(f"{min_row['description']}\t{min_row['reweighted_sc']}\n")

In [31]:
from utils_peptide.pdb_fixer import fixed_pdb_file
for file_entry in Path(result_path).iterdir():
    if file_entry.suffix == '.pdb':
        processed_pdb_path = Path(result_path) / file_entry.name
        candidate_pdb_path = Path(result_path) / f"{file_entry.stem.split('-')[0]}.pdb"
        candidate_pdb = []

        with open(processed_pdb_path) as f:
            lines = f.readlines()
            for line in lines:
                if line.startswith('ATOM') or line.startswith('TER') or line.startswith('END'):
                    candidate_pdb.append(line)

        with open(candidate_pdb_path, 'w') as f:
            f.write(''.join(candidate_pdb))

        fixed_pdb_file(candidate_pdb_path,result_path)